In [2]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import greycomatrix, greycoprops


# Mengambil path direktori dataset
data_dir = './training/'

# Mengambil path subdirektori untuk masing-masing kelas
healthy_dir = os.path.join(data_dir, 'daun_padi_sehat')
diseased_dir = os.path.join(data_dir, 'daun_padi_berpenyakit')

# Membuat daftar path gambar dan label kelas
image_paths = []
labels = []

# Menambahkan gambar dari direktori daun padi sehat ke daftar
for filename in os.listdir(healthy_dir):
    image_path = os.path.join(healthy_dir, filename)
    image_paths.append(image_path)
    labels.append(0)  # Label 0 untuk daun padi sehat

# Menambahkan gambar dari direktori daun padi berpenyakit ke daftar
for filename in os.listdir(diseased_dir):
    image_path = os.path.join(diseased_dir, filename)
    image_paths.append(image_path)
    labels.append(1)  # Label 1 untuk daun padi berpenyakit

# Mengonversi label ke bentuk numerik
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Mengonversi gambar ke bentuk vektor ciri (grayscale)
image_data = []
for image_path in image_paths:
    image = imread(image_path, as_gray=True)
    image = resize(image, (50, 50))  # Mengubah ukuran gambar menjadi 50x50 piksel
    image_data.append(image.flatten())

# Ekstraksi fitur warna (histogram warna)
def extract_color_features(image):
    # Ubah tipe data gambar menjadi uint8
    image_uint8 = (image * 255).astype(np.uint8)

    # Konversi gambar ke dalam model warna HSV
    hsv_image = cv2.cvtColor(image_uint8, cv2.COLOR_GRAY2BGR)
    hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2HSV)
    hist_hue = cv2.calcHist([hsv_image], [0], None, [256], [0, 256])
    hist_saturation = cv2.calcHist([hsv_image], [1], None, [256], [0, 256])
    hist_value = cv2.calcHist([hsv_image], [2], None, [256], [0, 256])
    cv2.normalize(hist_hue, hist_hue)
    cv2.normalize(hist_saturation, hist_saturation)
    cv2.normalize(hist_value, hist_value)
    color_features = np.concatenate([hist_hue.flatten(), hist_saturation.flatten(), hist_value.flatten()])
    return color_features


# Ekstraksi fitur tekstur (GLCM)
def extract_texture_features(image):
    glcm = greycomatrix(image.astype(np.uint8), [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast').ravel()
    energy = greycoprops(glcm, 'energy').ravel()
    homogeneity = greycoprops(glcm, 'homogeneity').ravel()
    texture_features = np.concatenate([contrast, energy, homogeneity])
    return texture_features

# Menggabungkan vektor fitur warna, tekstur, dan grayscale
final_features = []
for i in range(len(image_data)):
    gray_image = image_data[i].reshape(50, 50)  # Ubah kembali ke bentuk gambar grayscale
    color_features = extract_color_features(gray_image)
    texture_features = extract_texture_features(gray_image)
    combined_features = np.concatenate([image_data[i], color_features, texture_features])
    final_features.append(combined_features)

final_features = np.array(final_features)

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(final_features, labels, test_size=0.2, random_state=42)

# Membuat model SVM
model = SVC(kernel='linear')

# Melatih model dengan data latih
model.fit(X_train, y_train)

# Memprediksi label pada data uji
y_pred = model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi: {accuracy * 100:.2f}%')


c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in versi

Akurasi: 81.90%
